In [31]:
displaythis()

In [24]:
#load LDA scores
#update lda doc with scores - either one edge,
# or ALL that contain x IP and y port
def displaythis() :
    searches = widgets.Box(width=550,height=200)
    searchp = widgets.HBox(width=550,height=150)
    actions = widgets.Box(width=50,height=150)
    srcselect = widgets.Select(options=srclist,height=min(len(srclist)*10,150),width=120)
    dstselect = widgets.Select(options=dstlist,height=min(len(dstlist)*10,150),width=120)
    sportselect = widgets.Select(options=sportlist,height=min(len(sportlist)*10,150),width=75)
    dportselect = widgets.Select(options=dportlist,height=min(len(dportlist)*10,150),width=75)
    tableHTML = ("<table width='100%'><tr><td>" +
    "<td>Source IP:</td><td>Dest IP:</td><td>Src Port</td><td>Dst Port:</td></tr></table>")
    toplabel = widgets.HTML(value=tableHTML,width=550)
    ratingbut = widgets.RadioButtons(description='Rating:',options=['1', '2', '3'],width=50)
    updatebut = widgets.Button(description='Update')
    actions.children = [ratingbut,updatebut]
    searchp.children = [srcselect,dstselect,sportselect,dportselect,actions]
    searches.children = [toplabel,searchp]
    display(searches)
    
    def update_sconnects(b):
        #define logic based on combo of input
        src_ip,dst_ip = srcselect.value,dstselect.value
        sport,dport = sportselect.value,dportselect.value
        risk = ratingbut.value
        srchstr = '';
        if len(src_ip) > 0:
            srchstr += 'sip'
        if len(dst_ip) > 0:
            srchstr += 'dip'
        if len(sport) > 0:
            srchstr += 'sport'
        if len(dport) > 0:
            srchstr += 'dport'
        with open(tmpconnect,'w') as g:
            riter = csv.writer(g,delimiter=',')
            riter.writerow(["sev"]+["score"]+["Date"]+["srcIP"]+["dstIP"]+["SPort"]+["DPort"]+["Sfirst"]+["Dfirst"]+["Stotal"]+["Dtotal"]+["Spkts"]+["Dpkts"])
            with open(sconnect, 'r') as f:
                reader = csv.reader(f,delimiter=',')
                reader.next();
                rowct = 1
                #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
                for row in reader:
                    if srchstr == 'dipdport':
                        if row[4] == dst_ip and row[6] == dport:
                           row[0] = risk; 
                    elif srchstr == 'dip':
                        if row[4] == dst_ip:
                            row[0] = risk; 
                    elif srchstr == 'sipdip':
                        if row[3] == src_ip and row[4] == dst_ip:
                           row[0] = risk; 
                    elif srchstr == 'sipdipdport':
                        if row[3] == src_ip and row[4] == dst_ip and row[6] == dport:
                            row[0] = risk;
                    riter.writerow(row) 
                    rowct += 1
            !mv $tmpconnect $sconnect

    updatebut.on_click(update_sconnects)

In [19]:
import struct, socket
import numpy as np
import linecache, bisect
import csv
import operator
from IPython.html import widgets
from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display

spath = '/home/hadoop/ipython/vast/user/vast/'
sconnect = spath + 'lda_scores.csv'
sconnectbu = spath + 'lda_scores_bu.csv'
tmpconnect = sconnect+'.tmp'
stemp = sconnect + '.new'
coff = 150;
contxtpath = '/home/hadoop/ipython/vast/iploc/'
nwloc = contxtpath + 'networkcontext.txt'


In [3]:
# build dict of IP addresses
#sev,score, tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt
srcdict,srclist = {},[]
dstdict,dstlist = {},[]
sportdict,sportlist = {},[]
dportdict,dportlist = {},[]
srclist.append('')
dstlist.append('')
sportlist.append('')
dportlist.append('')
with open(sconnect, 'r') as f:
    reader = csv.reader(f,delimiter=',') 
    reader.next();
    rowct = 1
    for row in reader:
        if row[3] not in srcdict:
            srclist.append(row[3])
            srcdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
        if row[4] not in dstdict:
            dstlist.append(row[4])
            dstdict[row[4]] = struct.unpack("!L", socket.inet_aton(row[4]))[0]
        if row[5] not in sportdict:
            sportlist.append(row[5])
            sportdict[row[5]] = row[5]
        if row[6] not in dportdict:
            dportlist.append(row[6])
            dportdict[row[6]] = row[6]
        if rowct == coff:
            break;
        rowct += 1
iplist = np.loadtxt(nwloc,dtype=np.uint32,delimiter=',',usecols={0},
                    converters={0: lambda s: np.uint32(s.replace('"',''))})
#bisect - linecache
# iterate through IP addresses and attach ip location info
for srcip in srcdict:
        srcdict[srcip] = linecache.getline(nwloc, bisect.bisect(iplist,srcdict[srcip])).replace('\n','')
for dstip in dstdict:
        dstdict[dstip] = linecache.getline(nwloc, bisect.bisect(iplist,dstdict[dstip])).replace('\n','')

# print IP addresses
with open(sconnect, 'r') as f:
    reader = csv.reader(f,delimiter=',') 
    reader.next();
    rowct = 1
    for row in reader:
        #print "line",rowct,row,":"
        if srcdict[row[3]] == '':
            srcip = 'internet'
        else:
            srcip = srcdict[row[3]]
        if dstdict[row[4]] == '':
            dstip = 'internet'
        else:
            dstip = dstdict[row[4]]
            
        print 'line ',rowct,': ',srcip,"-->",dstip
        if rowct == 5:
            break
        rowct += 1

line  1 :  internet --> 2887647236,172.30.0.4,WEB03.BIGMKT3.COM
line  2 :  internet --> 2887647236,172.30.0.4,WEB03.BIGMKT3.COM
line  3 :  internet --> 2886336552,172.10.0.40,Administrator.BIGMKT1.COM
line  4 :  internet --> 2886336514,172.10.0.2,DC01.BIGMKT1.COM,DC
line  5 :  internet --> 2886336552,172.10.0.40,Administrator.BIGMKT1.COM


Occasionally, when you wish to update the suspicious connect list based on your review, run the cell below.

In [9]:
#sort LDA scores
# save list back
# refresh the diagram

with open(stemp,'w') as g:
    riter = csv.writer(g,delimiter=',')
    riter.writerow(["sev"]+["score"]+["Date"]+["srcIP"]+["dstIP"]+["SPort"]+["DPort"]+["Sfirst"]+["Dfirst"]+["Stotal"]+["Dtotal"]+["Spkts"]+["Dpkts"])

    reader = csv.reader(open(sconnect), delimiter=",")
    reader.next()
    srtlist = sorted(reader, key=operator.itemgetter(0,1))
    #for row in srtlist:
    riter.writerows(srtlist)
!mv $stemp $sconnect

In [134]:
#!cat $sconnect
!head -n 10 $sconnect

<h2>End Workflow</h2>

<b>Update Edge - to update a single edge use the code below.</b>

In [168]:
#enter the line number from above
#enter the new risk value - 1 for bad, 2 for hmmm, 3 for meh
line_no = 1;
risk = 2;

tmpconnect = sconnect+'.tmp'
with open(tmpconnect,'w') as g:
    riter = csv.writer(g,delimiter=',')
    riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+["dport"]+["proto"]+["ipkt"]+["ibyt"]+["lda_score"])
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',')
        reader.next();
        rowct = 1
        #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
        for row in reader:
            if rowct == line_no:
               row[0] = risk; 
            riter.writerow(row) 
            rowct += 1

!mv $tmpconnect $sconnect

To update all edges matching a particular condition, use the code below.

In [5]:
#only needed to reset demo
!cp $sconnectbu $sconnect
!head -n 10 $sconnect

In [11]:
# get network context - get start and end ranges
## need to write to updated file
tmpconnect = sconnect+'.tmp'
with open(tmpconnect,'w') as g:
    riter = csv.writer(g,delimiter=',')
    with open(nwloc, 'r') as f:
        reader = csv.reader(f,delimiter='\t') 
        #reader.next()
        nwdict = {}
        rowct = 1
        #address range, description
        for row in reader:
            if len(row) == 2:
                ipdec = [struct.unpack("!L", socket.inet_aton(row[0]))[0],row[0],row[1]]
            else:
                ipdec = [struct.unpack("!L", socket.inet_aton(row[0]))[0],row[0],row[1],row[2]]
            riter.writerow(ipdec)
            #dstdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
            rowct += 1



In [13]:
#!cp $tmpconnect $nwloc
!head -n 10 $nwloc